## Music Classification
This program finds the similarity between two songs by using __just the raw audiowave__ to compare with the accuracy of using song's features

In [11]:
from scipy.io.wavfile import read
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa

## Repeat to a Few Frames of Training Data
First we load the expected song similarity rating that is from 0-4. The rows and columns of the excel file are the names of the songs used. The value at row n and column m is the rating of how similar song n and song m are to each other. To eliminate repeated values and song pairs only the top diagonal of the matrix has values. The song similary between song 1 and song 2 is the same between song 2 and song 1 so there is nothing new to gain.

In [4]:
# load dataset that was created
import pandas as pd
df = pd.read_excel('Song Correlation Groups4.xlsx')

In [5]:
df.head()

,Nirvana - Heart Shaped Box,Alice in Chains - Rooster,Chic - Le Freak,Wild Cherry - Play that Funky Music,Chicago - 25 or 6 to 4,Green Day - Brain Stew,Becky G - Mayores,Maluma - Felices los 4,Kool & The Gang - Get Down On it,Gloria Gaynor - I Will Survive,...,Panic At The Disco - This Is Gospel,Blink-182 - All The Small Things,Good Charlotte - The Anthem,Enrique Iglesias - Subeme La Radio,Luis Fonsi - Echame La Culpa,Bee Gees - Night Fever,Bee Gees - You Should Be Dancing,Blondie - Heart of Glass,Lipps Inc - Funkytown,Daft Punk - Get Lucky
Nirvana - Heart Shaped Box,4.0,4.0,2.0,2.0,2.0,3.0,0.0,0.0,1.0,1.0,...,3.0,3.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2
Alice in Chains - Rooster,NaN,4.0,2.0,2.0,2.0,3.0,0.0,0.0,1.0,1.0,...,3.0,3.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2
Chic - Le Freak,NaN,NaN,4.0,4.0,1.0,1.0,1.0,1.0,4.0,4.0,...,1.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0,4.0,4
Wild Cherry - Play that Funky Music,NaN,NaN,NaN,4.0,2.0,1.0,1.0,1.0,4.0,4.0,...,1.0,1.0,1.0,2.0,2.0,4.0,4.0,4.0,4.0,4
Chicago - 25 or 6 to 4,NaN,NaN,NaN,NaN,4.0,3.0,0.0,0.0,0.0,0.0,...,3.0,2.0,2.0,1.0,1.0,3.0,3.0,3.0,3.0,3


In [6]:
# song titles
df.columns.values

array(['Nirvana - Heart Shaped Box', 'Alice in Chains - Rooster',
       'Chic - Le Freak', 'Wild Cherry - Play that Funky Music',
       'Chicago - 25 or 6 to 4', 'Green Day - Brain Stew',
       'Becky G - Mayores', 'Maluma - Felices los 4',
       'Kool & The Gang - Get Down On it',
       'Gloria Gaynor - I Will Survive', 'J Balvin - Ay Vamos',
       'Ozuna - Dile Que Tu Me Quieres',
       'Weezer - Undone -- The Sweater Song',
       'The Pixies - Where Is My Mind', 'The Smashing Pumpkins - Today',
       'Social Casualty - 5 Seconds Of Summer',
       "Fall Out Boy - Sugar, We're Goin Down",
       'Panic At The Disco - This Is Gospel',
       'Blink-182 - All The Small Things', 'Good Charlotte - The Anthem',
       'Enrique Iglesias - Subeme La Radio',
       'Luis Fonsi - Echame La Culpa', 'Bee Gees - Night Fever',
       'Bee Gees - You Should Be Dancing', 'Blondie - Heart of Glass',
       'Lipps Inc - Funkytown', 'Daft Punk - Get Lucky'], dtype=object)

In [7]:
# Frame size 
nfft = 22050

In [12]:
# Function to return a vector of raw audio
def getx(song):
    # load song
    y, sr = librosa.load("music files/" + song + ".wav")
    
    # take frames to do calculations of the middle of a song, after any intros
    nframe = 60
    ycrop = y[nframe*nfft:(nframe+40)*nfft]
    ycrop = np.ravel(ycrop)
    
    # Keep only a frame of the audio
    
    # tempo features. Beat_frames are the samples where a strong beat occurs
    #tempo, beat_frames = librosa.beat.beat_track(y=ycrop, sr=sr)
    # array beats is created so each song has the same length for x
    # beats = 0 unless a strong beat occurs at samples given from beat_frames
    #beats = np.zeros((ycrop.shape))  
    #beats[beat_frames] = 1            
    
    # percussive features. Will be output seperatly and joined later
    #y_percussive = librosa.effects.hpss(ycrop, margin=(1.0,5.0))[1]
    
    # chroma features averaged to represent the most common notes used
    #chroma_cq = librosa.feature.chroma_cqt(y=ycrop, sr=sr)
    #chroma_cq = np.mean(chroma_cq, axis = 1)
    #chroma_cq = np.ravel(chroma_cq) # flatten the 12 rows
    
    #S = librosa.feature.melspectrogram(y=ycrop, sr=sr, n_mels=128, fmax=8000)
    #S = np.ravel(S)
    
    # predictor features withour percussive elements
    #xi = np.hstack([sr, tempo, beats, y_percussive, chroma_cq])
    #xi = np.hstack([sr, tempo, chroma_cq, S])
    
    return ycrop#xi, y_percussive
    

In [13]:
# Load all songs and get song parameters
xr = []
#perc = []
for song in df.columns.values:
    print('loading song: ' + song)
    xi = getx(song)
    xr.append(xi)
    #perc.append(perci)

loading song: Nirvana - Heart Shaped Box
loading song: Alice in Chains - Rooster
loading song: Chic - Le Freak
loading song: Wild Cherry - Play that Funky Music
loading song: Chicago - 25 or 6 to 4
loading song: Green Day - Brain Stew
loading song: Becky G - Mayores
loading song: Maluma - Felices los 4
loading song: Kool & The Gang - Get Down On it
loading song: Gloria Gaynor - I Will Survive
loading song: J Balvin - Ay Vamos
loading song: Ozuna - Dile Que Tu Me Quieres
loading song: Weezer - Undone -- The Sweater Song
loading song: The Pixies - Where Is My Mind
loading song: The Smashing Pumpkins - Today
loading song: Social Casualty - 5 Seconds Of Summer
loading song: Fall Out Boy - Sugar, We're Goin Down
loading song: Panic At The Disco - This Is Gospel
loading song: Blink-182 - All The Small Things
loading song: Good Charlotte - The Anthem
loading song: Enrique Iglesias - Subeme La Radio
loading song: Luis Fonsi - Echame La Culpa
loading song: Bee Gees - Night Fever
loading song: B

In [18]:
# Turn to array
X = np.array(xr)
print(X.shape)
#Perc = np.array(perc)
#print(Perc.shape)

(27, 882000)


In [24]:
# Pickle to save x in case we expand it later
import pickle

with open("xr.p", "wb") as fp:
    pickle.dump(xr,fp)
    
#with open("perc.p", "wb") as fp:
#    pickle.dump(perc,fp)    

In [157]:
# Load X
with open("xr.p", "rb") as fp:
    xr = pickle.load(fp)
    
#with open("perc.p", "rb") as fp:
#    perc = pickle.load(fp)

In [14]:
# Y matrix listing likeness values (just extracted values from excel, not correct for neural network input)

y = np.array(df)
print(y[:10,:10])

[[  4.   4.   2.   2.   2.   3.   0.   0.   1.   1.]
 [ nan   4.   2.   2.   2.   3.   0.   0.   1.   1.]
 [ nan  nan   4.   4.   1.   1.   1.   1.   4.   4.]
 [ nan  nan  nan   4.   2.   1.   1.   1.   4.   4.]
 [ nan  nan  nan  nan   4.   3.   0.   0.   0.   0.]
 [ nan  nan  nan  nan  nan   4.   0.   0.   0.   0.]
 [ nan  nan  nan  nan  nan  nan   4.   4.   1.   1.]
 [ nan  nan  nan  nan  nan  nan  nan   4.   0.   0.]
 [ nan  nan  nan  nan  nan  nan  nan  nan   4.   4.]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan   4.]]


In [19]:
# Turn y into a vector

i = 0
j = 0
n = y.shape[0] #number of rows/columns
ind = 0

yvect = np.zeros((np.sum(range(n+1)),1)) 
order = np.zeros((np.sum(range(n+1)),2))

for i in range(n):
    for j in range(i,n):
        yvect[ind] = y[i,j]
        
        # to keep track of which song #s correspond to yvect indices
        order[ind] = np.array([i,j])
        ind = ind + 1

In [20]:
# Example of yvect and order

print('Target variable and corresponding song pair')
print('\ty\t\tsongs')
for i in range(10,20):
    print('\t%d'%yvect[i]+'\t\t%d, %d'%(order[i,0],order[i,1]))

Target variable and corresponding song pair
	y		songs
	0		0, 10
	0		0, 11
	3		0, 12
	3		0, 13
	3		0, 14
	3		0, 15
	3		0, 16
	3		0, 17
	3		0, 18
	3		0, 19


## Generating the predictors
The matrix X contains all the musical features of each song. y is the target variable for each possible song pair that is non-repeating. Therefore each corresponding predictor vector should contain the features from the two songs of that pair. For example, the song similary between song 1 and song 2 would have a predictor vector of both song 1 and 2 features stacked.

In [21]:
# Generate the final predictor vectors based on the song pairs
# This includes all the 'x' paramters and the cross correlation
# peaks of the percussive audio

ncomb = yvect.shape[0] #total number of non repeated song pairs
npeak = 1000 #number of peaks to save from standardized cross correlation
Xcomb = np.zeros((ncomb, X.shape[1]*2))
#Pcomb = np.zeros((ncomb, Perc.shape[1]*2))
for i in range(order.shape[0]):
    
    print('iteration %d of %d' %(i, order.shape[0]-1))
        
    # find cross correlation between the percussive elements of the two songs given by 'order'
    # Since drum beats are usually constant only a small porttion of correlation is found
    #xcorr = np.correlate(Perc[int(order[i,0]),:nfft*1],Perc[int(order[i,1]),:nfft*1], "same")
    #xcorst = (xcorr-np.mean(xcorr))/np.std(xcorr)
    #xcorpk = np.sort(xcorst)[-npeak:]
    
    # Xcomb takes the features from the two songs 
    Xcomb[i] = np.hstack((X[int(order[i,0]),:],  X[int(order[i,1]),:]))#, xcorpk))

iteration 0 of 377
iteration 1 of 377
iteration 2 of 377
iteration 3 of 377
iteration 4 of 377
iteration 5 of 377
iteration 6 of 377
iteration 7 of 377
iteration 8 of 377
iteration 9 of 377
iteration 10 of 377
iteration 11 of 377
iteration 12 of 377
iteration 13 of 377
iteration 14 of 377
iteration 15 of 377
iteration 16 of 377
iteration 17 of 377
iteration 18 of 377
iteration 19 of 377
iteration 20 of 377
iteration 21 of 377
iteration 22 of 377
iteration 23 of 377
iteration 24 of 377
iteration 25 of 377
iteration 26 of 377
iteration 27 of 377
iteration 28 of 377
iteration 29 of 377
iteration 30 of 377
iteration 31 of 377
iteration 32 of 377
iteration 33 of 377
iteration 34 of 377
iteration 35 of 377
iteration 36 of 377
iteration 37 of 377
iteration 38 of 377
iteration 39 of 377
iteration 40 of 377
iteration 41 of 377
iteration 42 of 377
iteration 43 of 377
iteration 44 of 377
iteration 45 of 377
iteration 46 of 377
iteration 47 of 377
iteration 48 of 377
iteration 49 of 377
iteration 

In [22]:
# Keras libraries

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [23]:
# Data Splitting

ntr = round((7/10)*ncomb)
Iperm = np.random.permutation(ncomb) #random permutation of all X sample rows
train = Iperm[:ntr]
test = Iperm[ntr:]
Xtr = Xcomb[train,:]
ytr = yvect[train]
Xts = Xcomb[test,:]
yts = yvect[test]

In [25]:
from keras.models import Model, Sequential
from keras.layers import Dense, Activation

In [26]:
# Keras Clear Session
import keras.backend as K
K.clear_session()

In [27]:
# Neural Network Model

nin = Xtr.shape[1]
nh = 50
nout = int(np.max(ytr+1)) 
model = Sequential()
model.add(Dense(nh, input_shape=(nin,), activation='sigmoid', name='hidden'))
model.add(Dense(nout, activation = 'softmax',name='output'))

In [28]:
# Print Model Summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 50)                88200050  
_________________________________________________________________
output (Dense)               (None, 5)                 255       
Total params: 88,200,305
Trainable params: 88,200,305
Non-trainable params: 0
_________________________________________________________________


In [29]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        # TODO:  Create two empty lists, self.loss and self.val_acc
        self.loss = []
        self.val_acc = []
 
    def on_batch_end(self, batch, logs={}):
        # TODO:  This is called at the end of each batch.  
        # Add the loss in logs.get('loss') to the loss list
        self.loss.append(logs.get('loss'))
        
    def on_epoch_end(self, epoch, logs):
        # TODO:  This is called at the end of each epoch.  
        # Add the test accuracy in logs.get('val_acc') to the val_acc list
        self.val_acc.append(logs.get('val_acc'))

# Create an instance of the history callback
history_cb = LossHistory()

In [30]:
from keras import optimizers

opt = optimizers.Adam(lr=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(Xtr, ytr, epochs=10, batch_size=100, validation_data=(Xts,yts), callbacks=[history_cb])

Train on 265 samples, validate on 113 samples
Epoch 1/10
265/265 [==============================] - 55s 208ms/step - loss: 1.9617 - acc: 0.3434 - val_loss: 1.7332 - val_acc: 0.3894
Epoch 2/10
265/265 [==============================] - 50s 187ms/step - loss: 1.4867 - acc: 0.4830 - val_loss: 1.6582 - val_acc: 0.4071
Epoch 3/10
265/265 [==============================] - 48s 181ms/step - loss: 1.3870 - acc: 0.5170 - val_loss: 1.6031 - val_acc: 0.4159
Epoch 4/10
265/265 [==============================] - 45s 169ms/step - loss: 1.3482 - acc: 0.5472 - val_loss: 1.5646 - val_acc: 0.4425
Epoch 5/10
265/265 [==============================] - 47s 176ms/step - loss: 1.3098 - acc: 0.5585 - val_loss: 1.5297 - val_acc: 0.4602
Epoch 6/10
265/265 [==============================] - 51s 192ms/step - loss: 1.2815 - acc: 0.5547 - val_loss: 1.5105 - val_acc: 0.4602
Epoch 7/10
265/265 [==============================] - 53s 198ms/step - loss: 1.2609 - acc: 0.5547 - val_loss: 1.4970 - val_acc: 0.4513
Epoch 8/1

In [32]:
# Find predicted y values from test data

yhatprob = model.predict(Xts)
yhat = np.argmax(yhatprob, axis =1)
print(yhat)

[4 2 4 2 1 0 0 3 2 3 4 0 2 4 3 0 1 1 0 0 4 0 1 0 0 2 3 4 1 3 0 3 3 4 0 0 4
 2 1 0 3 3 4 1 4 0 1 1 0 2 4 3 1 1 0 0 3 2 1 3 3 3 0 1 0 3 4 3 2 3 4 3 3 4
 4 0 0 4 0 4 4 0 2 0 3 0 3 0 2 2 0 4 3 2 4 0 0 4 4 0 4 2 1 3 1 2 0 3 0 1 0
 0 1]


In [34]:
# 1 where array value is 4 or 3 and 0 for other value
# A 3 or 4 is good enough to recommend a song to a user

yhat4 = np.ravel((((yhat == 4)) | (yhat == 3))*1)
yts4 = np.ravel(((yts == 4) | (yts == 3))*1)

yerr = yhat4 & yts4

print('The accuracy of finding a similar song: %0.3f'%(np.sum(yerr)/np.sum(yts4)))
# song pairs where yhat == 4 and yts = 4
yerrloc = np.where(yerr == 1)
yerrloc = np.ravel(yerrloc)

if (len(yerrloc) > 0):
    i = 0
    if (len(yerrloc) < 24):
        maxi = len(yerrloc)
    else:
        maxi = 24
    
    print('Target variable and corresponding song pair')
    print('\tsongs\t\ty\t\tyhat')
    while (i < maxi):
        print("\t%d, %d\t\t%d\t\t%d" %(order[test[yerrloc[i]]][0], order[test[yerrloc[i]]][1],yts[yerrloc[i]],
                                      yhat[yerrloc[i]]))
        i += 1
else:
    print("No such error found")

# order gives song index, test gives index used for testing order pairs
print('\nOne example of Songs that are recommended as similar\n%s & %s'%(
                            df.columns.values[int(order[test[yerrloc[0]]][0])], 
                            df.columns.values[int(order[test[yerrloc[0]]][1])]))

The accuracy of finding a similar song: 0.564
Target variable and corresponding song pair
	songs		y		yhat
	8, 24		4		4
	5, 15		3		3
	2, 9		4		4
	15, 15		4		3
	4, 26		3		3
	2, 2		4		4
	15, 16		4		3
	13, 18		4		3
	9, 9		4		4
	0, 16		3		3
	18, 18		4		3
	4, 15		3		3
	4, 16		3		3
	14, 16		3		3
	0, 15		3		3
	24, 24		4		4
	0, 13		3		3
	8, 25		4		4
	4, 4		4		3
	16, 17		4		4
	3, 23		4		4
	1, 5		3		3

One example of Songs that are recommended as similar
Kool & The Gang - Get Down On it & Blondie - Heart of Glass


In [48]:
# Falso positives
# subtract so 1 means yhat == 4 and yts != 4
#            -1 means yhat != 4 and yts == 4
yerr = yhat4 - yts4

# song pairs where yhat == 4 and yts != 4
yerrloc = np.where(yerr == 1)
yerrloc = np.ravel(yerrloc)

if (len(yerrloc) > 0):
    i = 0
    if (len(yerrloc) < 24):
        maxi = len(yerrloc)
    else:
        maxi = 24
    
    print('Target variable and corresponding song pair')
    print('\tsongs\t\ty\t\tyhat')
    while (i < maxi):
        print("\t%d, %d\t\t%d\t\t%d" %(order[test[yerrloc[i]]][0], order[test[yerrloc[i]]][1],yts[yerrloc[i]],
                                      yhat[yerrloc[i]]))
        i += 1
else:
    print("No such error found")
    

Target variable and corresponding song pair
	songs		y		yhat
	20, 23		0		4
	5, 25		1		4
	4, 8		0		3
	18, 26		1		4
	20, 26		0		4
	1, 4		2		4
	2, 13		2		4
	18, 25		1		4
	17, 25		1		4
	15, 24		1		4
	0, 9		1		4
	7, 15		0		3
	7, 14		0		4
	6, 19		0		4
	6, 15		0		3
	2, 18		2		4
	0, 7		0		4
	3, 15		1		3
	4, 6		0		3
	3, 4		2		4
	8, 19		1		4
	2, 4		1		4
	0, 2		2		4
	1, 7		0		4


In [49]:
# False negatives
# -1 means yhat != 4 and yts == 4
yerrloc = np.array(np.where(yerr == -1))
yerrloc = np.ravel(yerrloc)

if (len(yerrloc) > 0):
    i = 0
    if (len(yerrloc) < 24):
        maxi = len(yerrloc)
    else:
        maxi = 24
    print('Target variable and corresponding song pair')
    print('\tsongs\t\ty\t\tyhat')
    while (i < maxi):
        print("\t%d, %d\t\t%d\t\t%d" %(order[test[yerrloc[i]]][0], order[test[yerrloc[i]]][1],yts[yerrloc[i]],
                                      yhat[yerrloc[i]]))
        i += 1
else:
    print("No such error found")

Target variable and corresponding song pair
	songs		y		yhat
	5, 12		4		2
	23, 24		4		1
	0, 16		3		0
	12, 12		4		2
	22, 22		4		2
	3, 22		4		2
	13, 17		3		1
	6, 10		4		0
	7, 20		4		1
	2, 22		4		2
	11, 11		4		0
	1, 12		3		2
	14, 14		4		2
	9, 23		3		2
	6, 20		4		1
	22, 24		4		1
	12, 17		3		1
	11, 20		4		0
	4, 14		3		2
	16, 17		4		1


In [50]:
# Correctly labeled low values
# 1 where yhat and yts < 2 
yhat34 = np.ravel((yhat < 2)*1)
yts34 = np.ravel((yts < 2 )*1)

yerr = (yhat != yts) & (yhat34 & yts34)
yerrloc = np.array(np.where(yerr == -1))
yerrloc = np.ravel(yerrloc)

if (len(yerrloc) > 0):
    i = 0
    if (len(yerrloc) < 24):
        maxi = len(yerrloc)
    else:
        maxi = 24
    print('Target variable and corresponding song pair')
    print('\tsongs\t\ty\t\tyhat')
    while (i < maxi):
        print("\t%d, %d\t\t%d\t\t%d" %(order[test[yerrloc[i]]][0], order[test[yerrloc[i]]][1],yts[yerrloc[i]],
                                      yhat[yerrloc[i]]))
        i += 1
else:
    print("No such error found")

No such error found
